In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math
# from scipy.spatial import KDTree
from sklearn.neighbors import KDTree

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from multiprocessing import Pool

from itertools import product

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [5]:
def extend(submission,hits,limit=0.04, num_neighbours=18):

    df = submission.merge(hits,  on=['hit_id'], how='left')
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-90,90,1):

#         print ('\r %f'%angle, end='',flush=True)
        df1 = df.loc[(df.arctan2>(angle-1.5)/180*np.pi) & (df.arctan2<(angle+1.5)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<3: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1[['x', 'y', 'z']].values.T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        c = np.cos(a)
        s = np.sin(a)

        tree = KDTree(np.column_stack([c, s, r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]
            c0 = c[idx0]
            c1 = c[idx1]
            s0 = s[idx0]
            s1 = s[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0)

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1)

            ## extend start point
            ns = tree.query([[c0, s0, r0]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r0 - r[ns], a0 - a[ns])
            diff = 1 - np.cos(direction - direction0)
            ns = ns[(r0 - r[ns] > 0.01) & (diff < (1 - np.cos(limit)))]
            for n in ns: df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

            ## extend end point
            ns = tree.query([[c1, s1, r1]], k=min(num_neighbours, min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns] - r1, a[ns] - a1)
            diff = 1 - np.cos(direction - direction1)
            ns = ns[(r[ns] - r1 > 0.01) & (diff < (1 - np.cos(limit)))]
            for n in ns:  df.loc[df.hit_id == hit_ids[n], 'track_id'] = p

    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
# Change this according to your directory preferred setting
path_to_train = "./data/train_1"

# This event is in Train_1
event_prefix = "event000001000"

In [7]:
weights_0 = np.array([1.1, 1.1, .375, .25, 0.05, 0.05, 0])
weights_2 = np.array([1.05, 1.05, .37, .25, .01, .045, 0])
weights_3 = np.array([1.08, 1.08, .38,.25, 0.009, 0.0001, 0])
weights_4 = np.array([1.1, 1.1, .5, .25, 0.008, 0.001, 0])
weights_5 = np.array([1.01, 1.01, .4048, .2, 2e-16, 2e-16, 0])
weights_9 = np.array([1.02, 1.02, .38, .25, 0.009, 0.02, 0])

weights_1 = np.array([1.08, 1.08, .38,.25, 0.0001, 0.0001, 0])
weights_6 = np.array([1.08, 1.08, .38, .25, 2e-4, 0, 0])
weights_8 = np.array([1.07, 1.07, .37, .25, 0.0002, 0.0001, 0])
weights_12 = np.array([1.01, 1.01, .42, .25, 0.0023, 0.001, 0])
weights_13 = np.array([1.0, 1.0, .40, .20, 0.0023, 0.0001, 0])
weights_14 = np.array([1.02, 1.02, .40, .24, 0.0002, 0, 0])
weights_7 = np.array([1.08, 1.08, .40, .20, 0.0023, 0, 0])

weights_10 = np.array([1.02, 1.02, .42, .24, 0.0002, 0, 0])
weights_11 = np.array([1.08, 1.08, .40, .20, 0.0023, 0.0001, 0])
weights_15 = np.array([1.01, 1.01, .42, .25, 0.0002, 0, 0])
weights_16 = np.array([1.005, 1.005, .40, .25, 0.0002, 0, 0])
weights_17 = np.array([1.02, 1.02, .42, .24, 0.02, 0, 0])
weights_18 = np.array([1.033, 1.033, .376, .2, 0.0002, 0, 0])
weights_19 = np.array([0.9734, 0.9734, .4072, .2217, 0.0, 0.0, 0])
weights_20 = np.array([1.0, 1.0, .42, .24, 0.0, 0, 0])
weights_21 = np.array([1.02, 1.02, .42, .24, 0.0, 0, 0])
weights_22 = np.array([1.05, 1.05, .4, .25, 0.0, 0, 0])
weights_23 = np.array([1.05, 1.05, .5, .25, 0.0, 0, 0])
weights_24 = np.array([1.05, 1.05, .4, .20, 0.0, 0, 0])
weights_26 = np.array([1.2, 1.2, .45, .15, 0.0, 0, 0])
weights_27 = np.array([1.2, 1.2, .45, .20, 0.0, 0, 0])
weights_28 = np.array([1.2, 1.2, .45, .25, 0.0, 0, 0])
weights_29 = np.array([1.09, 1.09, .5, .20, 0.0, 0, 0])
weights_25 = np.array([1.09, 1.09, .45, .20, 0.0, 0, 0])
weights_30 = np.array([0.9, .9, .35, .15, 0.0, 0.007, 0.007])
weights_31 = np.array([1.09, 1.09, .45, .20, 0.0, 0.005, 0.005])
weights_34 = np.array([1.09, 1.09, .45, .20, 0.0, 0.01, 0.01])
weights_36 = np.array([1.0, 1.0, .35, .2, 0.0, 0.02, 0.02])
weights_32 = np.array([0.9, .9, .35, .2, 0.0, 0.01, 0.01])
weights_33 = np.array([1.0, 1.0, .35, .2, 0.0, 0.01, 0.01])
weights_37 = np.array([0.9, .9, .4, .2, 0.0, 0.015, 0.015]) # raise w4
weights_38 = np.array([0.9, .9, .4, .25, 0.0, 0.01, 0.01])  # raise w3
weights_39 = np.array([0.9, .9, .45, .2, 0.0, 0.01, 0.01])
weights_40 = np.array([0.95, .95, .4, .2, 0.0, 0.01, 0.01])
weights_41 = np.array([0.95, .95, .45, .2, 0.0, 0.01, 0.01])  # 38 + 40
weights_42 = np.array([0.95, .95, .4, .2, 0.0, 0.008, 0.008]) # w4 lowered
weights_43 = np.array([1.0, 1.0, .4, .2, 0.0, 0.01, 0.01])    # w1 raised
weights_44 = np.array([0.95, .95, .4, .2, 0.0, 0.012, 0.012]) # w4 raised
weights_45 = np.array([1.0, 1.0, .4, .2, 0.0, 0.01, 0.01])  # raise w1 from 35
weights_46 = np.array([0.9, .9, .4, .2, 0.0, 0.009, 0.009]) # lower w4
weights_47 = np.array([0.85, .85, .4, .2, 0.0, 0.01, 0.01]) # lower w1
weights_49 = np.array([0.9, .9, .4, .2, 0.0, 0.007, 0.007]) # still lower w4
weights_48 = np.array([0.9, .9, .4, .2, 0.0, 0.008, 0.008]) # lower w4
weights_51 = np.array([0.9, .9, .4, .2, 0.0, 0.009, 0.009]) # lower w4
weights_50 = np.array([0.9, .9, .4, .2, 0.0, 0.011, 0.011]) # higher w4
weights_52 = np.array([0.9, .9, .4, .2, 0.0, 0.012, 0.012]) # higher w4
weights_53 = np.array([0.9, .9, .4, .2, 0.0, 0.013, 0.013]) # higher w4
weights_54 = np.array([0.9, .9, .4, .2, 0.0, 0.014, 0.014]) # higher w4
weights_55 = np.array([0.9, .9, .4, .2, 0.0, 0.015, 0.015]) # higher w4
weights_56 = np.array([0.9, .9, .5, .2, 0.0, 0.01, 0.01])     # raise w2
weights_60 = np.array([0.7, .7, .5, .13, 0.0, 0.01, 0.01])     # raise w2, lower w1, w3
weights_59 = np.array([0.9, .9, .55, .1, 0.0, 0.01, 0.01])     # raise w2, lower w3
weights_35 = np.array([0.9, .9, .4, .2, 0.0, 0.01, 0.01])     
weights_57 = np.array([0.9, .9, .5, .15, 0.0, 0.01, 0.01]) # second best
weights_61 = np.array([0.9, .9, .4, .1, 0.0, 0.01, 0.01])  # lower w3 some more
weights_62 = np.array([0.9, .9, .45, .1, 0.0, 0.01, 0.01])  # lower w3 and raise w2
weights_63 = np.array([0.9, .9, .5, .1, 0.0, 0.01, 0.01])  # raise w2 some more
weights_64 = np.array([0.9, .9, .5, .07, 0.0, 0.01, 0.01])  # raise w2 and lower w3 some more
weights_65 = np.array([0.9, .9, .35, .15, 0.0, 0.01, 0.01])   # lower w2 a bit
weights_66 = np.array([0.9, .9, .35, .2, 0.0, 0.01, 0.01])   # lower w2 a bit and raise w3
weights_67 = np.array([0.95, .95, .3, .2, 0.0, 0.012, 0.012])# weights from r optimization
weights_68 = np.array([0.96, .96, .35, .2, 0.0, 0.0085, 0.0085])# weights from r optimization
weights_69 = np.array([0.9, .9, .4, .15, 0.0, 0.0085, 0.0085])   # w4 down
weights_70 = np.array([1.0, 1.0, .4, .1214, 0.0, 0.008, 0.008])   # w4 down
weights_71 = np.array([0.9, 0.9, .5, .15, 0.0, 0.0085, 0.0085])   # w4 down
weights_73 = np.array([0.9, .9, .45, .15, 0.0, 0.01, 0.01])   # w2 up a bit
weights_74 = np.array([0.9, .9, .35, .2, 0.0, 0.011, 0.011])   # w2 down and w3 up
weights_72 = np.array([0.9, .9, .4, .15, 0.0, 0.012, 0.012])   # w4 up a bit
weights_75 = np.array([0.9, .9, .4, .15, 0.0, 0.014, 0.014]) 
weights_76 = np.array([0.9, .9, .4, .20, 0.0, 0.014, 0.014]) 
weights_77 = np.array([0.85, 0.85, .4, .20, 0.0, 0.012, 0.012]) 
weights_78 = np.array([0.9, .9, .4, .15, 0.005, 0.01, 0.01]) 
weights_79 = np.array([0.9, .9, .4, .15, 0.0005, 0.01, 0.01]) 
weights_80 = np.array([0.9, .9, .4, .15, 0.01, 0.01, 0.01]) 
weights_82 = np.array([0.96, .96, .45, .126, 0.0, 0.007, 0.007])  
weights_84 = np.array([0.96, .96, .45, .15, 0.0, 0.0108, 0.0108])  
weights_86 = np.array([0.9, .9, .4, .126, 0.0, 0.0108, 0.0108])  
weights_85 = np.array([0.96, .96, .4, .15, 0.0, 0.0108, 0.0108])  
weights_87 = np.array([1, 1, .4, .13, 0.0, 0.0108, 0.0108])  
weights_88 = np.array([0.96, .96, .4, .13, 0.0, 0.0108, 0.0108])  

weights_58 = np.array([0.9, .9, .4, .15, 0.0, 0.01, 0.01])   # best so far
weights_81 = np.array([0.96, .96, .45, .126, 0.0, 0.0108, 0.0108])  
weights_83 = np.array([0.96, .96, .4, .126, 0.0, 0.0108, 0.0108])  
weights_89 = np.array([0.96, .96, .4, .2, 0.0, 0.005, 0.005])  



weights_arr = np.vstack([weights_0, weights_1, weights_2, weights_3, weights_4, weights_5, weights_6, weights_7, weights_8, weights_9, weights_10, weights_11, weights_12, weights_13, weights_14,  weights_15,  weights_16,  weights_17,  weights_18,  weights_19,  weights_20,  weights_21,  weights_22,  weights_23,  weights_24,  weights_25,  weights_26,  weights_27,  weights_28,  weights_29,  weights_30,  weights_31,  weights_32,  weights_33,  weights_34,  weights_35,  weights_36,  weights_37,  weights_38,  weights_39,  weights_40,  weights_41,  weights_42,  weights_43,  weights_44,  weights_45,  weights_46,  weights_47,  weights_48,  weights_49,  weights_50,  weights_51,  weights_52,  weights_53,  weights_54,  weights_55,  weights_56,  weights_57,  weights_58,  weights_59,  weights_60,  weights_61,  weights_62,  weights_63,  weights_64,  weights_65,  weights_66,  weights_67,  weights_68,  weights_69,  weights_70,  weights_71,  weights_72,  weights_73,  weights_74,  weights_75,  weights_76,  weights_77,  weights_78,  weights_79,  weights_80,  weights_81,  weights_82,  weights_83,  weights_84,  weights_85,  weights_86,  weights_87,  weights_88,  weights_89])

In [8]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import DBSCAN

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.5], eps=0.0034, dz0 = 0, stepdz = 1.5e-7, stepeps = 0.000655, num_loops=225, final_cluster=0, weight=81, weight_arr=weights_arr, max_size=16, step_z=0.000010, size_incr=0.8, min_points=1, max_cluster_size=21, final_cluster_size=5):
        self.max_cluster_size = max_cluster_size
        self.rz_scales=rz_scales
        self.epsilon = eps
        self.dz0 = dz0
        self.stepdz = stepdz
        self.stepeps = stepeps / num_loops
        self.num_loops = num_loops
        self.final_cluster = final_cluster
        self.weight_arr = weight_arr
        self.weights = weight
        self.max_size = max_size
        self.step_z = step_z
        if size_incr != 0:
            self.size_incr = (self.max_cluster_size - max_size) / (num_loops * size_incr)
        else:
            self.size_incr = 0
            
        self.min_points = min_points
        self.final_cluster_size = final_cluster_size
        
    # remove outliers
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 5
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    
    # not sure what this function does?
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    # standard scale our data
    def _preprocess(self, hits):
        # old preprocess, may work better?
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
        
    def _init(self,dfh):
        dfh['s1'] = dfh.hit_id
        dfh['N1'] =1
        dfh['stepped_z'] = dfh.z
        dfh['e1'] = self.epsilon
        dfh['group_locked'] = 0
        
        mm = 1
        dz0 = self.dz0
        
        cx = self.weight_arr[self.weights]
        if self.final_cluster:
            xy_weight = cx[5]
            xy_step = xy_weight / (self.num_loops * int(self.final_cluster))
        
        for ii in range(self.num_loops):
            # gradually reduce the weights for x2 and y2 since it is restricting the size of our clusters
            if (self.final_cluster) and (ii > self.num_loops // 2):
                cx[5] = cx[6] = xy_weight - (ii * xy_step)
            
            max_size = np.max([self.max_size + (ii * self.size_incr), self.max_cluster_size])
            
            dfh['r'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2+dfh['stepped_z'].values**2)
            dfh['rt'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2)
            dfh['a0'] = np.arctan2(dfh['y'].values,dfh['x'].values)
            dfh['z1'] = dfh['stepped_z'].values/dfh['rt'].values
            
            dfh['z2'] = dfh['stepped_z']/dfh['r']
            dfh['x2'] = dfh['x'].values/dfh['r'].values
            dfh['y2'] = dfh['y'].values/dfh['r'].values
            
            dfh['z3'] = 0
            mm = mm*(-1)  
            
            z_step = mm * self.step_z * ii
            dz = mm*((ii*self.stepdz))
            
            dfh['stepped_z'] = dfh['z'] + z_step    
            
            dfh['a1'] = dfh['a0']+mm*(dfh['rt']+(dz)*dfh['rt']**2)/1000*(ii/2)/180*math.pi
            dfh['sina1'] = np.sin(dfh['a1'].values)
            dfh['cosa1'] = np.cos(dfh['a1'].values)
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['sina1','cosa1','z1','z2', 'rt', 'x2', 'y2']].values)
            dfs = np.multiply(dfs, cx)

            clusters=DBSCAN(eps=self.epsilon+(ii*self.stepeps),min_samples=self.min_points,metric='euclidean',n_jobs=1).fit(dfs).labels_ 

            if ii==0:
                dfh['s1'] = clusters
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
                
            # else update our hits conditionally, if it's a better fit
            else:
                # put our new clusters to another feature
                dfh['s2'] = clusters
                dfh['e2'] = self.epsilon+(ii*self.stepeps)
                
                # get the count of those clusters
                dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                maxs1 = dfh['s1'].max()

                # if our new clusters are bigger, but less than our max size, use the new ones instead
                cond = np.where((dfh['N2'].values > dfh['N1'].values) & (dfh['N2'].values < max_size))
                
                s1 = dfh['s1'].values
                s1[cond] = dfh['s2'].values[cond]+maxs1
                
                # write the new clusters back to our dataframe
                dfh['s1'] = s1
                dfh['s1'] = dfh['s1'].astype('int64')
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            
        # for debugging
        # return dfh
        
        # return our clusters
        return dfh['s1'].values    
    
    def predict(self, hits):    
        # init our clusters
        self.clusters = self._init(hits) 
        
        mask = self.clusters == 0
        

        # preprocess our data
        X = self._preprocess(hits) 

        # create our last clusterer
        cl = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=self.final_cluster_size, metric='braycurtis', cluster_selection_method='leaf', algorithm='best', leaf_size=20)

        # labels = unique clusters
        labels = np.unique(self.clusters)

        # remove outliers
        self._eliminate_outliers(labels,X)

        # init n_labels
        n_labels = 0

        # now we loop through the points that haven't been assigned to a cluster and assign them with
        # HDBSCAN
        while n_labels < len(labels):
            n_labels = len(labels)
            max_len = np.max(self.clusters)
            mask = self.clusters == 0
        self.clusters[mask] = cl.fit_predict(X[mask])+max_len
        
        return self.clusters

In [9]:
path_to_test = "./data/test"
filename = "dbscan_outliers"
eps = 0.0035
dz0 = -0.00070
stepdz = 0.00001
stepeps = 0.000002

def one_loop(event_id):
    hits  = pd.read_csv(path_to_test + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(path_to_test + '/event%s-cells.csv'%event_id)
    print('Event ID: ', event_id)
                
    # Track pattern recognition 
    model = Clusterer()
    labels = model.predict(hits)

    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    
    for i in range(10): 
        one_submission = extend(one_submission, hits)
            
    one_submission.to_csv('./%09d.dbscan_outliers_tuned.csv.gz'%int(event_id), index=False, compression='gzip')
    
    return one_submission

def create_test_submissions(path_to_test = "./data/test", start=0, end=125):
    event_ids = [ '%09d'%i for i in range(start,end) ]

    pool = Pool(processes=16)
    results = pool.map(one_loop, event_ids)
    pool.close()
    
    return results

In [10]:
_ = create_test_submissions(start=0)

Event ID:  000000020
Event ID:  000000018
Event ID:  000000026
Event ID:  000000002
Event ID:  000000016
Event ID:  000000012
Event ID:  000000014
Event ID:  000000006
Event ID:  000000028
Event ID:  000000030
Event ID:  000000000
Event ID:  000000004
Event ID:  000000010
Event ID:  000000022
Event ID:  000000024
Event ID:  000000008


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


...

/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


In [11]:
event_ids = [ i for i in range(0,125) ]
submissions = []
for i,event_id in enumerate(event_ids):
    submission  = pd.read_csv('./%09d.dbscan_outliers_tuned.csv.gz'%event_id, compression='gzip')
    submissions.append(submission)

# Create submission file
submission = pd.concat(submissions, axis=0)
submission.to_csv('20180725_dbscan_optimized_w_extension.csv.gz', index=False, compression='gzip')
print(len(submission))

KeyboardInterrupt: 

In [ ]:
dataset_submissions = []
dataset_scores = []
for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=50, nevents=5):
    # Track pattern recognition
    model = Clusterer(eps=0.0035)
    labels = model.predict(hits)

    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    dataset_submissions.append(one_submission)

    # Score for the event
    score = score_event(truth, one_submission)
    dataset_scores.append(score)

    print("Score for event %d: %.8f" % (event_id, score))
print('Mean score: %.8f' % (np.mean(dataset_scores)))


Score for event 1050: 0.25598962



Score for event 1051: 0.25233863
